<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Требуется разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

## Загрузка данных

In [2]:
try:
    data = pd.read_csv('D:/YP_DS/insurance.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [3]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные не имеют пропусков и имеют корректные типы данных

## Умножение матриц

Создадим класс Линейной регрессии

In [5]:
# Создадим класс Линейной Регрессии
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y) # < напишите код здесь >
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$


\
Требуется доказать, что предсказания не изменятся, имеем  $a =  Xw$,   $a' = X'w'$
\
\
Доказательство:
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1}(X^T X)^{-1}P^{-T} P^T X^T y
$$
При перемножении : $$P^{-T} P^T = E$$
Получаем единичную матрицу и в итоге получаем : $$w' = P^{-1}(X^T X)^{-1} X^T y$$
$$
w' = P^{-1}w
$$
Подставим w' в исходное уравнения и получим:
$$
a' = XPP^{-1}w
$$
$$
a' = Xw
$$
$$
a = a'
$$

Для этого создим обучающие выборки и тестовые для подтверждения результатов модели

In [6]:
features = data.drop('Страховые выплаты',axis = 1)
target = data['Страховые выплаты']

In [7]:
train_features,test_features,train_target,test_target = train_test_split(features,target,test_size=0.25,random_state=12345)

In [8]:
model = LinearRegression()
model.fit(train_features, train_target)
predictions = model.predict(test_features)

In [9]:
r2_before = r2_score(test_target,predictions)
print('Метрика R2 до преобразований:', r2_before)

Метрика R2 до преобразований: 0.4352275712702667


Создадим рандомную обратимую матрицу с помощью Numpy и умножим её на наши исходные признаки, повторим процедуру.

In [10]:
rng = np.random.RandomState(12345)
rand_matrix = rng.rand(4,4)

Проверим матрицу на обратимость: Определитель матрицы != 0

In [11]:
np.linalg.det(rand_matrix) != 0

True

In [12]:
# Перемножаем матрицу признаков и обратную к ней матрицу
new_features = features@rand_matrix

In [13]:
# Заново разделим выборки только уже с новыми признаками
train_features,test_features,train_target,test_target = train_test_split(new_features,target,test_size=0.25,random_state=12345)

In [14]:
# Заново обучим модель используя новые признаки
model.fit(train_features,train_target)
predict = model.predict(test_features)

In [15]:
r2_after = r2_score(test_target,predict)
print('Метрика R2 после преобразований:',r2_after)

Метрика R2 после преобразований: 0.4352276639323065


In [16]:
print(f'Разница в процентном отношении: {(1-r2_after/r2_before)*100}%')

Разница в процентном отношении: -2.129048017884827e-05%


**Ответ:** Не изменится.

**Обоснование:** При перемножении матриц значения признаков построчно умножаются всё время на одни и те же числа, что изменяет величину признаков, но не изменяет их веса. Все признаки изменяют свои значения пропорционально друг другу.

## Алгоритм преобразования

**Алгоритм**

Для защиты данных пользователей умножим матрицу признаков на случайно сгенерированную обратимую матрицу , тем самым зашифровав исходные данные.

**Обоснование**

Опираясь на предыдущий шаг проекта, можно увидеть, что качество линейной регрессии не изменится после такого преобразования

## Проверка алгоритма

Используем предложенный алгоритм и преобразуем матрицу признаков

In [18]:
rng = np.random.RandomState(12345)
rand_matrix = rng.rand(4,4)

In [19]:
# Перемножаем матрицу признаков и обратную к ней матрицу
new_features = features@rand_matrix

In [20]:
# Заново разделим выборки только уже с новыми признаками
train_features,test_features,train_target,test_target = train_test_split(new_features,target,test_size=0.25,random_state=12345)

In [21]:
# Заново обучим модель используя новые признаки
model.fit(train_features,train_target)
predict = model.predict(test_features)

In [22]:
r2_after = r2_score(test_target,predict)
print('Метрика R2 после преобразования данных пользователей:',r2_after)

Метрика R2 после преобразования данных пользователей: 0.4352276639323065


Метрики "До" и "После" преобразований не отличаются

## Вывод

Разработан такой метод преобразования данных, что по ним сложно восстановить персональную информацию.\
Метод корректно работает, так как метрики качества модели до преобразования и после преобразования не поменялись